In [1]:
# 分组级运算和转换
import numpy as np
import pandas as pd
import statsmodels.api as sm
from pandas import DataFrame, Series

In [2]:
df = DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                'key2' : ['one', 'two', 'one', 'two', 'one'],
                'data1' : np.random.randn(5),
                'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,1.818085,-0.660265
1,a,two,0.335633,-0.159298
2,b,one,-1.527032,1.420667
3,b,two,1.330872,0.370962
4,a,one,0.654796,2.366077


In [3]:
k1_means = df.groupby('key1').mean().add_prefix('mean_')
k1_means

,mean_data1,mean_data2
key1,,
a,0.936171,0.515505
b,-0.098080,0.895815


In [4]:
pd.merge(df, k1_means, left_on='key1', right_index=True)

,key1,key2,data1,data2,mean_data1,mean_data2
0,a,one,1.818085,-0.660265,0.936171,0.515505
1,a,two,0.335633,-0.159298,0.936171,0.515505
4,a,one,0.654796,2.366077,0.936171,0.515505
2,b,one,-1.527032,1.420667,-0.098080,0.895815
3,b,two,1.330872,0.370962,-0.098080,0.895815


In [5]:
people = DataFrame(np.random.randn(5, 5),
                   columns=['a', 'b', 'c', 'd', 'e'],
                   index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people

,a,b,c,d,e
Joe,-0.172694,-0.709400,-0.081344,1.209596,-0.172941
Steve,-0.209070,-1.433533,-0.389780,2.624777,-0.458535
Wes,-0.741516,1.276627,-0.754021,-0.969686,-0.764511
Jim,-1.837946,0.195744,0.704963,-1.964799,-0.111329
Travis,-0.700793,1.479144,-0.429784,0.193810,-1.155268


In [6]:
key = ['one', 'two', 'one', 'two', 'one'] # 每一行的名称
people.groupby(key).mean()

,a,b,c,d,e
one,-0.538334,0.682124,-0.421716,0.144573,-0.697573
two,-1.023508,-0.618895,0.157591,0.329989,-0.284932


In [7]:
people.groupby(key).transform(np.mean) # transform变回原来形状，但是用前面分组聚合的值填空。

,a,b,c,d,e
Joe,-0.538334,0.682124,-0.421716,0.144573,-0.697573
Steve,-1.023508,-0.618895,0.157591,0.329989,-0.284932
Wes,-0.538334,0.682124,-0.421716,0.144573,-0.697573
Jim,-1.023508,-0.618895,0.157591,0.329989,-0.284932
Travis,-0.538334,0.682124,-0.421716,0.144573,-0.697573


In [8]:
def demean(arr):
    return arr - arr.mean()
demeaned = people.groupby(key).transform(demean) # 
demeaned

,a,b,c,d,e
Joe,0.365641,-1.391524,0.340372,1.065023,0.524632
Steve,0.814438,-0.814639,-0.547372,2.294788,-0.173603
Wes,-0.203182,0.594504,-0.332305,-1.114260,-0.066938
Jim,-0.814438,0.814639,0.547372,-2.294788,0.173603
Travis,-0.162459,0.797020,-0.008068,0.049237,-0.457694


In [9]:
demeaned.groupby(key).mean() # 因为前面每个值都减去了平均值，所以应该是0。显示不为0是因为浮点数计算误差。

,a,b,c,d,e
one,0.000000e+00,0.000000e+00,-5.551115e-17,0.0,3.700743e-17
two,1.110223e-16,-5.551115e-17,0.000000e+00,0.0,2.775558e-17


In [10]:
# apply：一般性的“拆分－应用－合并”

In [11]:
tips = pd.read_csv('tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill'] # 新加一列，小费与账单金额的比例。
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [12]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:] # 获取小费比例最高的n条数据
top(tips, n=3)

,total_bill,tip,smoker,day,time,size,tip_pct
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [13]:
tips.groupby('smoker').apply(top, n=2, column='tip_pct') # 先按是否吸烟分组，再分别查看小费比例。

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [14]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [15]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [16]:
result.unstack('smoker') # 把列添加到索引，作用在smoker外。

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [17]:
result.apply(lambda x: x.describe()) # result.describe()

,count,mean,std,min,25%,50%,75%,max
count,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
mean,122.000000,0.161262,0.062514,0.046217,0.121838,0.154735,0.190036,0.501167
std,41.012193,0.002735,0.031968,0.014961,0.021309,0.001258,0.007103,0.295822
min,93.000000,0.159328,0.039910,0.035638,0.106771,0.153846,0.185014,0.291990
25%,107.500000,0.160295,0.051212,0.040928,0.114305,0.154291,0.187525,0.396578
50%,122.000000,0.161262,0.062514,0.046217,0.121838,0.154735,0.190036,0.501167
75%,136.500000,0.162229,0.073817,0.051507,0.129372,0.155180,0.192547,0.605756
max,151.000000,0.163196,0.085119,0.056797,0.136906,0.155625,0.195059,0.710345


In [18]:
# 禁止分组键

In [19]:
tips.groupby('smoker', group_keys=False).apply(top) # 禁止构成多重索引

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [20]:
# 分位数和桶分析

In [21]:
frame = DataFrame({'data1': np.random.randn(1000),
                   'data2': np.random.randn(1000)})
factor = pd.cut(frame.data1, 4) # 切4份
factor[:5] # 前5个元素所在区间

0    (-1.822, -0.128]
1     (-0.128, 1.566]
2    (-1.822, -0.128]
3     (-0.128, 1.566]
4    (-1.822, -0.128]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.523, -1.822] < (-1.822, -0.128] < (-0.128, 1.566] < (1.566, 3.26]]

In [22]:
def get_stats(group):
    return {'min': group.min(),
             'max': group.max(),
             'count': group.count(),
             'mean': group.mean()}
grouped = frame.data2.groupby(factor) # 根据data1的分段，对data2进行分组。
result = grouped.apply(get_stats)
print(result)
print(result.unstack())

data1                  
(-3.523, -1.822]  min       -1.848876
                  max        2.181951
                  count     38.000000
                  mean      -0.026754
(-1.822, -0.128]  min       -3.228809
                  max        2.746706
                  count    408.000000
                  mean       0.050160
(-0.128, 1.566]   min       -2.686684
                  max        2.661151
                  count    504.000000
                  mean       0.016858
(1.566, 3.26]     min       -1.925868
                  max        1.666792
                  count     50.000000
                  mean      -0.098734
Name: data2, dtype: float64
                       min       max  count      mean
data1                                                
(-3.523, -1.822] -1.848876  2.181951   38.0 -0.026754
(-1.822, -0.128] -3.228809  2.746706  408.0  0.050160
(-0.128, 1.566]  -2.686684  2.661151  504.0  0.016858
(1.566, 3.26]    -1.925868  1.666792   50.0 -0.098734


In [23]:
grouping = pd.qcut(frame.data1, 10, labels=list('ABCDEFGHIJ')) # False的话就是默认数字编号
grouping.head() # 返回每个元素的区间编号

0    A
1    G
2    C
3    G
4    B
Name: data1, dtype: category
Categories (10, object): [A < B < C < D ... G < H < I < J]

In [24]:
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
A,-3.228809,2.539322,100.0,-0.015510
B,-2.185821,2.176634,100.0,-0.017168
C,-2.019923,2.746706,100.0,0.164830
D,-1.893227,2.284246,100.0,-0.058456
E,-1.896332,2.402686,100.0,0.035064
F,-2.686684,2.297081,100.0,0.098516
G,-2.291797,1.819254,100.0,-0.063452
H,-2.240964,2.518475,100.0,0.192147
I,-2.154790,2.661151,100.0,-0.063368


In [25]:
# 示例：用特定于分组的值填充缺失值

In [26]:
s = Series(np.random.randn(6))
s[::2] = np.nan
s.fillna(s.mean()) # 用平均数填充缺失值

0    0.130493
1   -0.152785
2    0.130493
3   -0.107106
4    0.130493
5    0.651370
dtype: float64

In [27]:
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4 # 前4个东部州，后4个西部州。
data = Series(np.random.randn(8), index=states)
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio          2.202163
New York     -0.068105
Vermont            NaN
Florida       1.734117
Oregon        0.438046
Nevada             NaN
California    0.068062
Idaho              NaN
dtype: float64

In [28]:
data.groupby(group_key).mean() # 非NA求平均值

East    1.289392
West    0.253054
dtype: float64

In [29]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean) # 分组之后用每组的平均值填充

Ohio          2.202163
New York     -0.068105
Vermont       1.289392
Florida       1.734117
Oregon        0.438046
Nevada        0.253054
California    0.068062
Idaho         0.253054
dtype: float64

In [30]:
fill_values = {'East': 0.5, 'West': -1} # 指定填充值
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio          2.202163
New York     -0.068105
Vermont       0.500000
Florida       1.734117
Oregon        0.438046
Nevada       -1.000000
California    0.068062
Idaho        -1.000000
dtype: float64

In [31]:
# 示例：随机采样和排列

In [32]:
# 红桃（Hearts）、黑桃（Spades）、梅花（Clubs）、方片（Diamonds）
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4 # Python3下range是生成器，必须用list显示展开。
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)
deck = Series(card_val, index=cards)
deck.head() # 牌面数组

AH    1
2H    2
3H    3
4H    4
5H    5
dtype: int64

In [33]:
def draw(deck, n=5): # 随机抽n张
    return deck.take(np.random.permutation(len(deck))[:n])
draw(deck)

QC     10
10D    10
4S      4
6S      6
9D      9
dtype: int64

In [34]:
# 每种花色中随机抽取两张牌
get_suit = lambda card: card[-1]
deck.groupby(get_suit).apply(draw, n=2) # 默认根据索引排序，索引的最后一个字符是花色。

C  JC     10
   5C      5
D  7D      7
   4D      4
H  8H      8
   10H    10
S  3S      3
   7S      7
dtype: int64

In [35]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2) # 效果一样，但是不用多重索引。

9C     9
7C     7
8D     8
QD    10
4H     4
AH     1
JS    10
8S     8
dtype: int64

In [36]:
# 示例：分组加权平均数和相关系数

In [37]:
df = DataFrame({'category': ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'],
                'data': np.random.randn(8),
                'weights': np.random.rand(8)})
df

,category,data,weights
0,a,-0.792357,0.249365
1,a,0.288613,0.669163
2,a,-1.106979,0.713345
3,a,0.730819,0.408806
4,b,-2.450098,0.928670
5,b,-0.440074,0.272057
6,b,-0.397476,0.576002
7,b,-0.083851,0.795196


In [38]:
grouped = df.groupby('category')

In [39]:
get_wavg = lambda g: np.average(g['data'], weights=g['weights']) # 求加权平均，weights自动归一化处理。
grouped.apply(get_wavg) # 分组计算

category
a   -0.242739
b   -1.046175
dtype: float64

In [40]:
close_px = pd.read_csv('stock_px.csv', parse_dates=True, index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5472 entries, 1990-02-01 to 2011-10-14
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AA      5472 non-null   float64
 1   AAPL    5472 non-null   float64
 2   GE      5472 non-null   float64
 3   IBM     5472 non-null   float64
 4   JNJ     5472 non-null   float64
 5   MSFT    5472 non-null   float64
 6   PEP     5471 non-null   float64
 7   SPX     5472 non-null   float64
 8   XOM     5472 non-null   float64
dtypes: float64(9)
memory usage: 427.5 KB


In [41]:
close_px.head()

,AA,AAPL,GE,IBM,JNJ,MSFT,PEP,SPX,XOM
1990-02-01,4.98,7.86,2.87,16.79,4.27,0.51,6.04,328.79,6.12
1990-02-02,5.04,8.00,2.87,16.89,4.37,0.51,6.09,330.92,6.24
1990-02-05,5.07,8.18,2.87,17.32,4.34,0.51,6.05,331.85,6.25
1990-02-06,5.01,8.12,2.88,17.56,4.32,0.51,6.15,329.66,6.23
1990-02-07,5.04,7.77,2.91,17.93,4.38,0.51,6.17,333.75,6.33


In [42]:
rets = close_px.pct_change().dropna() # 扔掉有空数据的行
spx_corr = lambda x: x.corrwith(x['SPX']) # 与SPX的相关系数
by_year = rets.groupby(lambda x: x.year) # 指定用那个函数去做group
by_year.apply(spx_corr) # 按年分组并计算与SPX的相关系数

,AA,AAPL,GE,IBM,JNJ,MSFT,PEP,SPX,XOM
1990,0.595024,0.545067,0.752187,0.738361,0.801145,0.586691,0.783168,1.0,0.517586
1991,0.453574,0.365315,0.759607,0.557046,0.646401,0.524225,0.641775,1.0,0.569335
1992,0.398180,0.498732,0.632685,0.262232,0.515740,0.492345,0.473871,1.0,0.318408
1993,0.259069,0.238578,0.447257,0.211269,0.451503,0.425377,0.385089,1.0,0.318952
1994,0.428549,0.268420,0.572996,0.385162,0.372962,0.436585,0.450516,1.0,0.395078
1995,0.291532,0.161829,0.519126,0.416390,0.315733,0.453660,0.413144,1.0,0.368752
1996,0.292344,0.191482,0.750724,0.388497,0.569232,0.564015,0.421477,1.0,0.538736
1997,0.564427,0.211435,0.827512,0.646823,0.703538,0.606171,0.509344,1.0,0.695653
1998,0.533802,0.379883,0.815243,0.623982,0.591988,0.698773,0.494213,1.0,0.369264
1999,0.099033,0.425584,0.710928,0.486167,0.517061,0.631315,0.336593,1.0,0.315383


In [43]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT'])) # 计算两个股票之间的相关系数

1990    0.408271
1991    0.266807
1992    0.450592
1993    0.236917
1994    0.361638
1995    0.258642
1996    0.147539
1997    0.196144
1998    0.364106
1999    0.329484
2000    0.275298
2001    0.563156
2002    0.571435
2003    0.486262
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

In [44]:
# 示例：面向分组的线性回归

In [45]:
def regress(data, yvar, xvars):
    y= data[yvar]
    x = data[xvars]
    x['intercept'] = 1.
    result = sm.OLS(y, x).fit()
    return result.params

by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
1990,1.512772,0.001395
1991,1.187351,0.000396
1992,1.832427,0.000164
1993,1.390470,-0.002657
1994,1.190277,0.001617
1995,0.858818,-0.001423
1996,0.829389,-0.001791
1997,0.749928,-0.001901
1998,1.164582,0.004075
1999,1.384989,0.003273
